## 02 – Daily log returns

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
prices_path = Path("../data/prices_2018_2024.parquet")
assert prices_path.exists(), f"Missing input file: {prices_path}"

prices = pd.read_parquet(prices_path)
prices = prices.sort_index()

In [3]:
prices = prices.where(prices > 0)

log_returns = np.log(prices / prices.shift(1))
log_returns = log_returns.dropna()

In [4]:
assert not log_returns.empty

In [5]:
log_returns_path = Path("../data/log_returns_2018_2024.parquet")
log_returns_path.parent.mkdir(parents=True, exist_ok=True)

log_returns.to_parquet(log_returns_path)